### 3. Find regions of interest, related topics, type from google trends

In [1]:
import re
import numpy as np
import time

In [2]:
import pandas as pd                        
from pytrends.request import TrendReq
#proxies=['https://34.203.233.13:80',]
pytrends = TrendReq(hl='en-US', tz=530, timeout=(10,25), retries=2, backoff_factor=0.1)

In [3]:
file_name = 'step2_trendy_data.pkl'
df = pd.read_pickle(file_name)

In [4]:
df.head()

,title,popularity
0,#DubbakaBypolls,99.000000
1,#IrandamKuththu,77.141566
2,Bihar Assembly,63.786451
3,NEET results,48.965465
4,#BiharElectionResults,47.482541


#### Finding type

In [5]:
titletype = {}
for topic in df['title']:
    keywords = pytrends.suggestions(keyword=topic)
    #print(keywords)
    try:
        
        if not topic.startswith('hashtag'):
            title = keywords[0]['title']
        else:
            title = topic
        type_of = keywords[0]['type']
    except IndexError:
        title = topic
        type_of = 'hashtag'
    titletype[topic] = type_of
    
    
    

In [6]:
df['Type'] = df['title'].map(titletype)

In [7]:
df.head()

,title,popularity,Type
0,#DubbakaBypolls,99.000000,hashtag
1,#IrandamKuththu,77.141566,hashtag
2,Bihar Assembly,63.786451,Topic
3,NEET results,48.965465,Entrance examination
4,#BiharElectionResults,47.482541,hashtag


#### Finding Related Topics

In [8]:
titles = df['title']

related_titles = {}
hashtags = []

for title in titles:
    if title.startswith('#'):
        hasht = title
        hashtags.append(title)
        title = title[1:]
        title = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', title)
        if len(title)<1: 
            continue
        title = ' '.join(title)
        try:
            pytrends.build_payload(kw_list=[title])
            #print(title)
            related_topic = pytrends.related_topics()
            related = list(related_topic[list(related_topic.keys())[0]]['rising']['topic_title'])
            related_titles.update({hasht: related[:4]})
        except Exception as e:
            print(title)
    else:
        try:
            pytrends.build_payload(kw_list=[title])
            #print(title)
            related_topic = pytrends.related_topics()
            related = list(related_topic[list(related_topic.keys())[0]]['rising']['topic_title'])
            related_titles.update({title: related[:4]})
        except Exception as e:
            print(e)
            #print(title,related_topic)
        
        

Dubbaka Bypolls
IPL Unacademy Video


In [9]:
#related_titles

In [10]:
df['Related Topics'] = df['title'].map(related_titles)

In [11]:
df

,title,popularity,Type,Related Topics
0,#DubbakaBypolls,99.000000,hashtag,NaN
1,#IrandamKuththu,77.141566,hashtag,"[Kuththu, Film, Irandam Ulagam, Download]"
2,Bihar Assembly,63.786451,Topic,"[Kasturbhai Lalbhai, Constituent Assembly of I..."
3,NEET results,48.965465,Entrance examination,"[2017, NEET · 2017, NEET · 2019, NEET · 2018]"
4,#BiharElectionResults,47.482541,hashtag,"[2017, Amethi, United States presidential elec..."
5,bighit,46.416561,Company,"[GFriend, Seventeen, Concert tour, Blackpink]"
6,Shikhar Dhawan,43.898400,Indian cricketer,"[Hardik Pandya, Rishabh Pant, 2019 ICC Cricket..."
7,Jaipur,42.561668,City in India,"[Sapna Choudhary, Dupatta, Jaipur Pink Panther..."
8,Sira,40.754191,City in India,"[Needham, Sye Raa Narasimha Reddy, Somerville,..."
9,Starc,35.165562,Food,"[2018 Indian Premier League, 2016 Indian Premi..."


#### Finding Region of Interest

In [12]:
regions_of_interest = {}

for title in titles:
    hasht = title
    if title.startswith('#'):
        hasht = title
        title = title[1:]
        title = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', title)
        if len(title)<1: 
            continue
        title = ' '.join(title)
        
    
    kw_list = [title]
    pytrends.build_payload(kw_list, cat=0, timeframe='today 5-y', geo='IN', gprop='')
    regions = pytrends.interest_by_region(resolution='DMA', inc_low_vol=True, inc_geo_code=False)
    regions['region'] = regions.index
    regions = regions.to_numpy()
    regions = sorted(regions, key=lambda regions: regions[0]) [::-1]
    
    regions_of_interest.update({hasht:regions})

        
    print(title)
    time.sleep(5)


Dubbaka Bypolls
Irandam Kuththu
Bihar Assembly
NEET results
Bihar Election Results
bighit
Shikhar Dhawan
Jaipur
Sira
Starc
Boycott Amazon
EVMs
Exit
Mouse Hunt
Bigil
Bihar Result
Jharkhand
Sushil Modi
Captain Marvel
Kabir Singh
Avengers: Endgame
Lok Sabha Elections
Cricket World Cup
IPL Unacademy Video
Article 370
Harbhajan Singh
India Today
Joker
Chandrayaan 2
PM Kisan Yojana
Congress
Owaisi
Yogendra Yadav
Burnol
NDTV
Mozambique
World Science Day
Rona
Mahagathbandhan
Nitish
Armenia
Only 20%
Bihar Elections
good air movement


In [13]:
df['regions_of_interest'] = df['title'].map(regions_of_interest)

In [14]:
df

,title,popularity,Type,Related Topics,regions_of_interest
0,#DubbakaBypolls,99.000000,hashtag,NaN,"[[0, West Bengal], [0, Uttarakhand], [0, Uttar..."
1,#IrandamKuththu,77.141566,hashtag,"[Kuththu, Film, Irandam Ulagam, Download]","[[100, Tamil Nadu], [59, Kerala], [19, Karnata..."
2,Bihar Assembly,63.786451,Topic,"[Kasturbhai Lalbhai, Constituent Assembly of I...","[[100, Bihar], [61, Jharkhand], [48, Tripura],..."
3,NEET results,48.965465,Entrance examination,"[2017, NEET · 2017, NEET · 2019, NEET · 2018]","[[100, Andhra Pradesh], [63, Jammu and Kashmir..."
4,#BiharElectionResults,47.482541,hashtag,"[2017, Amethi, United States presidential elec...","[[100, Bihar], [65, Daman and Diu], [54, Dadra..."
5,bighit,46.416561,Company,"[GFriend, Seventeen, Concert tour, Blackpink]","[[100, Mizoram], [48, Nagaland], [45, Arunacha..."
6,Shikhar Dhawan,43.898400,Indian cricketer,"[Hardik Pandya, Rishabh Pant, 2019 ICC Cricket...","[[100, Daman and Diu], [88, Jharkhand], [83, G..."
7,Jaipur,42.561668,City in India,"[Sapna Choudhary, Dupatta, Jaipur Pink Panther...","[[100, Rajasthan], [9, Haryana], [8, Delhi], [..."
8,Sira,40.754191,City in India,"[Needham, Sye Raa Narasimha Reddy, Somerville,...","[[100, Chandigarh], [81, Karnataka], [58, Punj..."
9,Starc,35.165562,Food,"[2018 Indian Premier League, 2016 Indian Premi...","[[100, Tripura], [88, Goa], [86, Jammu and Kas..."


In [15]:
df.to_pickle('step3_data.pkl')